# Search with Encrypted Query

In this tutorial, we will walk through the steps to use the ES2 with encrypted query.

If data for query are also needed to be secured, we can use CC (Cipher Query - Cipher DB) search.

For example:
- To retrieve data from a database with restricted access
    + Users should verify that their encrypted security identifier is contained within the encrypted list of authorized privileges

## Import ES2

To use the ES2 SDK, you need to install it first.

Before installing, make sure you have conda installed on your system.

For more details, see `SDK installation` section in `Get Started`.

After installation, you can import the ES2 SDK in your Python code.

In [ ]:
# !pip install es2
!pip install --force-reinstall sdk/python/dist/es2-1.0.2-cp312-cp312-linux_x86_64.whl

In [ ]:
import es2

## Initialize ES2

To use the ES2 service, initialization is required. 

Initialization step includes
1) Establishing a connection to the `ES2` server
2) Configuring Crypto settings necessary for vector search
3) Registering evaluation keys for enabling `ES2` server to perform secure operations.

You can set the path and ID of the key for data encryption, presets for operations, query encryption, database encryption, and index type.

In [ ]:
KEYPATH = "./keys"
KEYID = "cc_tutorial_key"
DIM=512

In [ ]:
es2.init(
    host="localhost",
    port=40050,
    key_path=KEYPATH,
    key_id=KEYID,
)

## Prepare Data

### Prepare Plaintext Vectors

Let's generate random data set first, which will be secured, encrypted DB

In [ ]:
import numpy as np

# Define a function to generate random vectors
def generate_random_vector(dim):
    if dim <= 16 or dim > 4096:
        raise ValueError(f"Invalid dimension: {dim}.")
    
    vec = np.random.uniform(-1.0, 1.0, dim)
    norm = np.linalg.norm(vec)

    if norm > 0:
        vec = vec / norm

    return vec

# Prepare Data
num_data = 10

db_vectors = [
    generate_random_vector(512) for _ in range(num_data)
]
db_metadata = [f"data_{i+1}" for i in range(num_data)]

## Create Index and Insert Data

In previous tutorial, we know that for encrypted similarity search, we first prepare a vector index, called `Index`.

To use encrypted query for search, we should set configuration of index as available for encrypted query.


In [ ]:
index = es2.create_index("cc_search_index", dim=DIM, query_encryption="cipher") # Create index with query encryption config as cipher

Below is also available.
```python
# First, create index normally
index = es2.create_index("cc_search_index", dim=DIM)
# And then, set configuration for query encryption
index.index_config.query_encryption = "cipher"
```

If the index is ready, you can insert data into it.

As we know, this first **encrypts the vectors** using the generated encryption keys and **inserts** them into the index in the created ES2.

In [ ]:
index.insert(db_vectors, metadata=db_metadata)

## Encrypted Similarity Search

### Prepare query

First, prepare query for encrypted search.

For now, let's use first data we generated randomly as query.

In [ ]:
query_vector = db_vectors[0]

### Encrypted search on the index

Let's perform search with encrypted query for encrypted RAG. 

As we already set configuration of index to use encrypted query, plain query input will automatically be encrypted in process.

This process ensures secure and efficient similarity search operations, even when working with encrypted data.

In [ ]:
result = index.search(query_vector, top_k=2, output_fields=["metadata"])[0]
result

Of course, we can encrypt query with our own hands.

In this case, we can create new `Cipher` object or use that index already have.

For now, let's use one that index object have.

When we encrypt query ourselves, we should notice that `encode_type` value is `query` (Because default setting is for DB items)

In [ ]:
cipher = es2.Cipher(f"{KEYPATH}/{KEYID}/EncKey.bin", dim=DIM)
encrypted_query_vector = cipher.encrypt(query_vector, encode_type="query")
result = index.search(encrypted_query_vector, top_k=2, output_fields=["metadata"])[0]
result

### Clean Up

In [ ]:
es2.drop_index("cc_search_index")

In [ ]:
es2.release_key("cc_tutorial_key")